# Datasheet analysis

Perform all the router datasheet analysis presented in the IMC'25 paper (Section 3)

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Initialization

In [ ]:

# Select the different output format settings

PaperPlot = True
# PaperPlot = False
if PaperPlot:
    output_format = 'IMC'
else:
    output_format = 'online'

if output_format == 'online':
    font_size_px = 14
    linewidth_px = 512
    landscapewidth_px = 654
    plot_path = None
    
    plot_path = Path('plots')

if output_format == 'IMC':
    font_size_pt = 7
    offset = 5 # to compensate for the rounding of unit conversions
    linewidth_pt = 241 - offset  
    landscapewidth_pt = 506 - offset
    
    # 1pt = 1.333px
    font_size_px = int(font_size_pt*1.333)+1
    linewidth_px = int(linewidth_pt*1.333)+1
    landscapewidth_px = int(landscapewidth_pt*1.333)+1

    out_path = Path('output/2025_IMC/figures')

# Create the output directory if don't exist
Path(out_path).mkdir(parents=True, exist_ok=True)

# Input data
input_path = Path('input')

# Default plot layout
default_layout = {
    "title":None,
    "width":linewidth_px,
    "height":200,
    "font":{"size":font_size_px},
    "yaxis":{'title':{'font':{'size':font_size_px}}},
    "xaxis":{'title':{'font':{'size':font_size_px},
                      'text':'Time [ s ]'}}
}


In [ ]:
# ==============
# Analysis metadata
# ==============

date_start  = '2024-09-01'
date_end    = '2024-11-01'

# Adjust the granularity at which we want to aggregate the data points
resampling  = '30min' 

## Aggregated power values

In [ ]:
# ==============
# Plot the total power and traffic time series
# ==============

# .. load the whole thing
df_ts = pd.read_csv(Path(input_path,'traffic_volume_power.csv'),index_col='timestamp', parse_dates=True)

# .. clean up
filter = df_ts.index.to_series().between(date_start, date_end)
df_ts = df_ts[filter]
resampling = '2h'
df_ts = df_ts.resample(resampling).mean()

#.. convert the units
df_ts['Tbps'] = df_ts['traffic volume [Bytes]']*8/1e12

# .. prepare the yaxis labels
total_capacity_Tbps = 74.9 
traffic_ticks = [1,2]
traffic_ticks_with_relative = ['{} ({}%)'.format(i,round(i/total_capacity_Tbps*100,1)) for i in traffic_ticks]
traffic_ticks_with_relative.insert(0,'0')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(
            x=df_ts.index,
            y=df_ts['power [W]'],
            name='Total power',
            line=dict(width=1),
        ),
            secondary_y=False,
)
fig.add_trace(go.Scatter(
            x=df_ts.index,
            # y=df_ts['traffic volume [Bytes]'],
            y=df_ts['Tbps'],
            name='Total traffic',
            line=dict(width=1),
        ),
            secondary_y=True,
)


 # y axis title
ytitle1 = go.layout.Annotation(
        x=-0.01,
        y=1.15,
        xref="paper",
        yref="paper",
        text="Power (W)",
        showarrow=False,
        xanchor='left'
    )
ytitle2 = go.layout.Annotation(
        x=0.95,
        y=1.15,
        xref="paper",
        yref="paper",
        text="Traffic (Tbps (%))",
        showarrow=False,
        xanchor='right'
    )

# Define the custom layout options
custom_layout = dict(
    xaxis=dict(
        tickangle=30,
    ),
    yaxis1=dict(
        autorange=True
    ),
    yaxis2=dict(
        # title=None,
        range=[0,2.45],
        tickmode = 'array',
        tickvals = [0, 1, 2],
        ticktext = traffic_ticks_with_relative
    ),
    legend=dict(
        xanchor="center",
        x=0.5,
        yanchor="bottom",
        y=0,
        orientation='h'
    ),
    annotations=[ytitle1,ytitle2],
    margin=dict(l=0, r=0, t=25, b=0),
    xaxis_tickformat = '%b %d'
)
# Combine with the defaults and apply
layout = default_layout.copy()
layout.update(custom_layout)
fig.update_layout(layout)

fig.show()
fig.write_image(out_path/'totalPowerTraffic.pdf')

In [ ]:

# ===============
# Compute the median power draw for the whole network
# ===============

df_ts['power [W]'].median()

In [ ]:
# ===============
# Computing the marginal cost of sending traffic
# ===============

def pretty_print(packet_size_B,traffic_cost,pkt_cost,bit_cost):
    print('''Packet size:\t{} Bytes
Packet cost:\t{:.2f} W
Bit cost:\t{:.2f} W
== 
Total cost:\t{:.2f} W
          
          '''.format(int(packet_size_B),pkt_cost,bit_cost,traffic_cost))
    return 0

Ebit = 5e-12
Epkt = 15e-9

# Computing the cost for 100G of traffic
bit_rate = 100e9

# .. 1500B packets
print('For 100Gbps')
packet_size_B = 1500
packet_size_b = packet_size_B*8
pkt_rate = bit_rate/packet_size_b

pkt_cost = pkt_rate*Epkt
bit_cost = bit_rate*Ebit
traffic_cost = pkt_cost + bit_cost
pretty_print(packet_size_B,traffic_cost,pkt_cost,bit_cost)

# .. 64B packets
print('For 100Gbps')
packet_size_B = 64
packet_size_b = packet_size_B*8
pkt_rate = bit_rate/packet_size_b

pkt_cost = pkt_rate*Epkt
bit_cost = bit_rate*Ebit
traffic_cost = pkt_cost + bit_cost
pretty_print(packet_size_B,traffic_cost,pkt_cost,bit_cost)

# Computing for Switch
print('For the entire Switch traffic')
pkt_rate = df_ts['packets'].median()
bit_rate = (df_ts['traffic volume [Bytes]'].median())*8

pkt_cost = pkt_rate*Epkt
bit_cost = bit_rate*Ebit
packet_size_B = bit_rate/pkt_rate/8
traffic_cost = pkt_cost + bit_cost
pretty_print(packet_size_B,traffic_cost,pkt_cost,bit_cost)

# .. compare to the average power
total_power = df_ts['power [W]'].median()
traffic_cost_rel = traffic_cost/total_power*100
print('Percentage of traffic cost wrt the total router power:')
print(traffic_cost_rel)


In [ ]:
# ===============
# Load the PSU data
# ===============

src_file = Path(input_path,'psu-efficiency.csv')
df = pd.read_csv(src_file)

model_ids = df['router_model'].unique()

model_data = {
    '8201-32FH': {
        'max_power' : np.nan,
        'typical_power' : 288,
    },
    '8201-24H8FH': {
        'max_power' : np.nan,
        'typical_power' : 205,
    },
    'ASR-9001': {
        'max_power' : 525,
        'typical_power' : 425,
    },
    'ASR-920-24SZ-M': {
        'max_power' : 145,
        'typical_power' : 110,
    },
    'N540-24Z8Q2C-M': {
        'max_power' : 250,
        'typical_power' : 200,
    },
    'NCS-55A1-24H': {
        'max_power' : 800,
        'typical_power' : 600,
    },
    'NCS-55A1-24Q6H-SS': {
        'max_power' : 550,
        'typical_power' : 400,
    },
    'NCS-55A1-48Q6H': {
        'max_power' : 700,
        'typical_power' : 460,
    },
    'WS-C6504-E': {
        'max_power' : np.nan,
        'typical_power' : np.nan,
    }
}


for m in sorted(model_ids):
    tmp = df.loc[df['router_model']== m].copy()
    tmp['total_power'] = tmp['median_power_PSU1']+tmp['median_power_PSU2']
    average_power = tmp['total_power'].median()
    model_data[m]['router_id'] = m
    model_data[m]['avg_power'] = average_power
    model_data[m]['relative_to_typical'] = 100 - average_power/model_data[m]['typical_power'] * 100 

# model_data

for m in sorted(model_ids):
    if 'WS-C6504-E' in m:
        continue
    model_tag = '\\model{'+m+'}'
    if model_data[m]['relative_to_typical'] < 0:
        relative_tag = (
            '\\textbf{\\color{red}'+
            str(int(model_data[m]['relative_to_typical']))+
            '}'
        )
    else:
        relative_tag =int(model_data[m]['relative_to_typical'])
            
    print(
          model_tag, '& ',
          int(model_data[m]['avg_power']), '\\,W'
          '&',
          model_data[m]['typical_power'], '\\,W'
          '&',
          relative_tag, '\\,\\%'
          '\\\\',
          )


In [ ]:
# Reconstruct the Broadcom plot

broadcom_file = Path(input_path,'broadcom_eff=f(t).csv')
df_bc = pd.read_csv(broadcom_file)

fig = px.line(
    df_bc,
    x = 'year',
    y = 'efficiency',
    markers=True
)

 # y axis title
ytitle = go.layout.Annotation(
        x=-0.01,
        y=1.15,
        xref="paper",
        yref="paper",
        text="Efficiency (W / 100 Gbps)",
        showarrow=False,
        xanchor='left'
    )

# Define the custom layout options
custom_layout = dict(
    xaxis=dict(
        title=dict(
            text='Year',
            font={'size':font_size_px},
        ),
    ),
    yaxis=dict(
        title=None,
        range=[0,30],
    ),
    legend=dict(
        orientation='v'
    ),
    annotations=[ytitle],
    margin=dict(l=0, r=0, t=25, b=0),
    width=0.47*linewidth_px
)
# Combine with the defaults and apply
layout = default_layout.copy()
layout.update(custom_layout)
fig.update_layout(layout)

fig.show()
fig.write_image(out_path/'trend_broadcom.pdf')

In [ ]:
# Generate the equivalent plot as the Broadcom one

df = pd.read_csv(Path(input_path,'datasheet.csv'))
display(df)

# Filter for models that have at least 100G of max throughput
# -> for smaller values, the efficiency metric does not make sense
df = df.loc[df['max_throughput_value']>100]
df['efficiency'] = df['power_draw_value']/(df['max_throughput_value']/100)

fig = go.Figure(
    go.Scatter(
        x = df['release_date'],
        y = df['efficiency'],
        mode='markers',
        marker_opacity=0.3
    ))

 # y axis title
ytitle = go.layout.Annotation(
        x=-0.01,
        y=1.15,
        xref="paper",
        yref="paper",
        text="Efficiency (W / 100 Gbps)",
        showarrow=False,
        xanchor='left'
    )

# Define the custom layout options
custom_layout = dict(
    xaxis=dict(
        title=dict(
            text='Year',
            font={'size':font_size_px},
        ),
    ),
    yaxis=dict(
        title=None,
        range=[0,100], # Filter out two outliers that make the plot hard to read
    ),
    legend=dict(
        orientation='v'
    ),
    annotations=[ytitle],
    margin=dict(l=0, r=5, t=25, b=0),
    width=0.47*linewidth_px
)
# Combine with the defaults and apply
layout = default_layout.copy()
layout.update(custom_layout)
fig.update_layout(layout)

fig.show()
fig.write_image(out_path/'trend_datasheet.pdf')